### Introduction

Our company is entering the film industry by launching a new movie studio but lacks experience in film production. We are tasked with analyzing current box office trends to identify successful film types. The goal is to translate these insights into actionable recommendations that will guide the studio head in creating films that align with market preferences and drive success.

### Problem Statement
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

### Objectives
1. Evaluate the performance of various film genres in both domestic and international markets.
2. Identify the movie studios that consistently produce high-performing films.
3. Investigate the correlation between production budgets and both domestic and international revenues.
4. Examine trends in film production over time.
5. Assess the impact of a film's popularity score and average rating on its overall performance.

### Data
Datasets used were obtained from:
* [Box Office Mojo](https://www.boxofficemojo.com/)
* [IMDB](https://www.imdb.com/)
* [Rotten Tomatoes](https://www.rottentomatoes.com/)
* [TheMovieDB](https://www.themoviedb.org/)
* [The Numbers](https://www.the-numbers.com/)


### Exploratory Data Analysis

In [105]:
# import the necessary packages
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm

#### Loading the Files Both (C.S.V) and (T.S.V)

In [106]:
# lets import the necessary files
movie_gross_df = pd.read_csv('data/bom.movie_gross.csv.gz')
movie_info_df = pd.read_csv('data/rt.movie_info.tsv.gz',delimiter='\t')
movie_reviews_df = pd.read_csv('data/rt.reviews.tsv.gz',delimiter='\t',encoding='latin-1')
movies_df = pd.read_csv('data/tmdb.movies.csv.gz',index_col=0)
movie_budget_df = pd.read_csv('data/tn.movie_budgets.csv.gz')


conn = sqlite3.connect('data/im.db')


# connecting to the database
conn = sqlite3.connect('data/im.db')
cursor = conn.cursor()
cursor.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
table_name = cursor.fetchall()
table_name

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [107]:
# check the first few rows 
movie_gross_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [108]:
movie_gross_df.shape

(3387, 5)

In [109]:
movie_gross_df.isna().sum().sort_values(ascending = False)

foreign_gross     1350
domestic_gross      28
studio               5
title                0
year                 0
dtype: int64

In [110]:
movie_gross_df1 = movie_gross_df.copy(deep = True)

In [111]:
# lets drop the rows with missing values.
movie_gross_df1.dropna(subset = ['studio', 'domestic_gross'], inplace =True)

# convert the values in the foreign_gross to a float
movie_gross_df1['foreign_gross'] = movie_gross_df1['foreign_gross'].str.replace(',','')
movie_gross_df1['foreign_gross'] = pd.to_numeric(movie_gross_df1['foreign_gross'])
movie_gross_df1['foreign_gross'].fillna(movie_gross_df1['foreign_gross'].median(), inplace = True)

movie_gross_df1.isna().sum().sort_values(ascending = False)

title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

# cleaning movie_info_df

In [112]:
movie_info_df.isna().sum().sort_values(ascending = False)

currency        1220
box_office      1220
studio          1066
writer           449
theater_date     359
dvd_date         359
director         199
synopsis          62
runtime           30
genre              8
rating             3
id                 0
dtype: int64

In [113]:
# Lets split the genre column
movie_info_df['genre_split1'] = movie_info_df['genre'].str.split('|').str[0]
movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,genre_split1
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN,Action and Adventure
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One,Drama
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN,Drama
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN,Drama
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN,Drama


In [114]:
# select the relevant columns for the analysis
movie_info_df = movie_info_df[['id', 'rating', 'genre_split1', 'runtime', 'studio']]
movie_info_df = movie_info_df.rename(columns = {'genre_split1':'genre'}) 

In [115]:
movie_info_df.isna().sum().sort_values(ascending = False)

studio     1066
runtime      30
genre         8
rating        3
id            0
dtype: int64

In [116]:
movie_info_df1 = movie_info_df.copy(deep = True)

In [117]:
movie_info_df1.shape

(1560, 5)

In [118]:
# Lets drop the rows with missing values..
movie_info_df1.dropna(subset = ['genre', 'rating', 'runtime'], inplace = True)

# assign the missing values in director a placeholder (Not Specified)
movie_info_df1['studio'].fillna('Not Specified', inplace = True)

In [119]:
movie_info_df1.isna().sum().sort_values(ascending = False)

id         0
rating     0
genre      0
runtime    0
studio     0
dtype: int64

# checking on reviews_df

In [120]:
movie_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [121]:
movies_df.isna().sum().sort_values(ascending = False)

genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [122]:
movies_df1 = movies_df.copy(deep = True)

In [123]:
movies_df1 = movies_df1[['id','popularity', 'title', 'vote_average', 'vote_count']]
movies_df1.head()

,id,popularity,title,vote_average,vote_count
0,12444,33.533,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,10191,28.734,How to Train Your Dragon,7.7,7610
2,10138,28.515,Iron Man 2,6.8,12368
3,862,28.005,Toy Story,7.9,10174
4,27205,27.920,Inception,8.3,22186


In [124]:
movie_budget_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [125]:
movie_budget_df1 = movie_budget_df.copy(deep = True)

In [126]:
movie_budget_df1['production_budget'] = pd.to_numeric(movie_budget_df1['production_budget'].astype(str).str.replace('[^\d]', '', regex = True))

movie_budget_df1['domestic_gross'] = pd.to_numeric(movie_budget_df1['domestic_gross'].astype(str).str.replace('[^\d]', '', regex = True))
  
movie_budget_df1['worldwide_gross'] =  pd.to_numeric(movie_budget_df1['worldwide_gross'].astype(str).str.replace('[^\d]', '', regex = True))


In [127]:
movie_budget_df1.isna().sum().sort_values(ascending = False)

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [128]:
movie_budget_df1.drop('release_date', axis = 1, inplace = True)

In [129]:
movie_budget_df1.head()

,id,movie,production_budget,domestic_gross,worldwide_gross
0,1,Avatar,425000000,760507625,2776345279
1,2,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,Dark Phoenix,350000000,42762350,149762350
3,4,Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


In [130]:
print(movie_gross_df1.duplicated().sum())
print(movie_info_df1.duplicated().sum())
print(movie_budget_df1.duplicated().sum())
print(movies_df1.duplicated().sum())

0
0
0
1020


In [131]:
movies_df1.drop_duplicates(inplace = True)
print(movies_df1.duplicated().sum())

0


# Cleaning the im.db 

In [132]:
 # lets access the movie_basics data from the database..
query1 = """SELECT * FROM movie_basics;"""
movie_basics_df = pd.read_sql(query1, conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [133]:
movie_basics_df.isna().sum().sort_values(ascending = False)

runtime_minutes    31739
genres              5408
original_title        21
movie_id               0
primary_title          0
start_year             0
dtype: int64

In [134]:
movie_basics_df.shape

(146144, 6)

In [135]:
movie_basics_df1 = movie_basics_df.copy(deep = True)

In [136]:
movie_basics_df1.dropna(subset = ['runtime_minutes', 'genres', 'original_title'], inplace = True)

In [137]:
movie_basics_df1.isna().sum().sort_values(ascending = False)

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
dtype: int64

In [138]:
# Lets split the genre column
movie_basics_df1['genre_split'] = movie_basics_df1['genres'].str.split(',').str[0]
movie_basics_df1.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,genre_split
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Action
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Biography
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",Comedy
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy,Comedy


In [139]:
#movie_basics_df1.rename(columns={'genre_split' : 'genres_1'},inplace=True)

In [140]:
movie_basics_df1.head()


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,genre_split
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Action
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Biography
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,Drama
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",Comedy
5,tt0111414,A Thin Life,A Thin Life,2018,75.0,Comedy,Comedy


In [141]:
movie_basics_df1.drop(['genres', 'start_year'], axis = 1, inplace = True)


In [142]:
movie_basics_df1.head()

,movie_id,primary_title,original_title,runtime_minutes,genre_split
0,tt0063540,Sunghursh,Sunghursh,175.0,Action
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,114.0,Biography
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,122.0,Drama
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,80.0,Comedy
5,tt0111414,A Thin Life,A Thin Life,75.0,Comedy


In [143]:
movie_basics_df1.rename(columns={'genre_split' : 'genres'},inplace=True)

In [144]:
movie_basics_df1.head()

,movie_id,primary_title,original_title,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,175.0,Action
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,114.0,Biography
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,122.0,Drama
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,80.0,Comedy
5,tt0111414,A Thin Life,A Thin Life,75.0,Comedy
